In [16]:
import serial
import time
from datetime import datetime
from IPython.display import clear_output


In [17]:
# —— 串口设置 ——————————————————————————————
port        = 'COM8'
baudrate    = 115200
bytesize    = serial.EIGHTBITS    # 8 位数据
parity      = serial.PARITY_NONE  # 无校验
stopbits    = serial.STOPBITS_ONE # 1 位停止
timeout     = 0.1                 # 非阻塞超时（秒）
xonxoff     = False               # 关闭 XON/XOFF 流控
rtscts      = False               # 关闭 RTS/CTS 流控
dsrdtr      = False               # 关闭 DSR/DTR 流控

# —— 读取周期 & 空闲自动退出设置 —————————————————————
read_interval  = 1.0   # 每隔 1 秒 检查一次串口并打印
idle_timeout   = 5.0   # 如果连续 5 秒都没读到任何数据，就自动退出

# 打开串口
ser = serial.Serial(
    port=port,
    baudrate=baudrate,
    bytesize=bytesize,
    parity=parity,
    stopbits=stopbits,
    timeout=timeout,
    xonxoff=xonxoff,
    rtscts=rtscts,
    dsrdtr=dsrdtr
)
print(f"Opened {ser.port} @ {ser.baudrate}bps, 8N1, flowctrl=None")

Opened COM8 @ 115200bps, 8N1, flowctrl=None


In [18]:
from datetime import datetime

last_data_time = time.time()

def decode_angle(msg: str):
    """
    msg 格式形如 'A+216' 或 'A-17911'，
    返回浮点角度，单位度（°）。
    """
    try:
        raw = int(msg[1:])      # msg[1:] 包含符号和数字，比如 '+216' 或 '-17911'
        return raw / 100.0       # 除以 100 转成真正的度数
    except ValueError:
        return None              # 解析失败时返回 None

try:
    while True:
        # 空闲超时检测
        if time.time() - last_data_time > idle_timeout:
            print(f"\n⚠️ 已连续 {idle_timeout} 秒无数据，自动停止读取。")
            break

        n = ser.in_waiting
        if n:
            raw_bytes = ser.read(n)
            last_data_time = time.time()

            text = raw_bytes.decode('ascii', errors='replace').strip()
            # 按行处理，防止一次读到多条消息
            for line in text.splitlines():
                if not line:
                    continue
                ts = datetime.now().strftime("%H:%M:%S.%f")[:-3]
                angle = decode_angle(line)
                if angle is not None:
                    # 带符号保留两位小数
                    print(f"[{ts}] {angle:+.2f}°")
                else:
                    # 无法解码则原样输出
                    print(f"[{ts}] {line}")

        time.sleep(read_interval)

except Exception as e:
    print("\n❌ 发生异常，停止读取：", e)

finally:
    ser.close()
    print("串口已关闭。")

[17:06:56.879] +2.66°
[17:06:57.879] +2.69°
[17:06:57.879] +2.69°
[17:06:57.879] +2.66°
[17:06:57.879] +2.70°
[17:06:57.879] +2.66°
[17:06:57.879] +2.69°
[17:06:57.879] +2.66°
[17:06:57.879] +2.70°
[17:06:57.879] +2.69°
[17:06:57.879] +2.69°
[17:06:57.879] +2.69°
[17:06:57.879] +2.69°
[17:06:57.879] +2.69°
[17:06:57.879] +2.70°
[17:06:57.879] +2.69°
[17:06:57.879] +2.70°
[17:06:57.879] +2.69°
[17:06:57.879] +2.66°
[17:06:57.879] +2.69°
[17:06:57.879] +2.66°
[17:06:57.879] +2.69°
[17:06:57.879] +2.69°
[17:06:57.879] +2.70°
[17:06:57.879] +2.66°
[17:06:57.879] +2.69°
[17:06:57.879] +2.66°
[17:06:57.879] +2.69°
[17:06:57.879] +2.69°
[17:06:57.879] +2.69°
[17:06:57.879] +2.66°
[17:06:57.879] +2.69°
[17:06:57.879] +2.69°
[17:06:57.879] +2.69°
[17:06:57.879] +2.69°
[17:06:57.879] +2.69°
[17:06:57.879] +2.69°
[17:06:57.879] +2.69°
[17:06:57.879] +2.69°
[17:06:57.879] +2.70°
[17:06:57.879] +2.69°
[17:06:57.879] +2.69°
[17:06:57.879] +2.69°
[17:06:57.879] +2.69°
[17:06:57.879] +2.69°
[17:06:57.

KeyboardInterrupt: 